In [3]:
from dataclasses import dataclass
from typing import Any, Optional, TypedDict
from urllib.parse import urljoin

import requests
import urllib3
from bs4 import BeautifulSoup

# ignore InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

ROOT = "https://www.riss.kr/"
PATH = "search/Search.do"
param_dict = {
  "isDetailSearch": 'N',
  "searchGubun": True,
  "viewYn": 'OP',
  "strQuery": '엣지 디바이스',
  "order": '/DESC',
  "onHanja": False,
  "strSort": 'RANK',
  "iStartCount": 0,
  "fsearchMethod": 'search',
  "sflag": 1,
  "isFDetailSearch":'N',
  "pageNumber": 1,
  "icate": 're_a_kor',
  "colName": 're_a_kor',
  "pageScale": 10,
  "isTab": 'Y',
  "query": '엣지 디바이스',
}


HeaderType = TypedDict("HeaderType", {"User-Agent": str, "Referer": str})


@dataclass(frozen=True)
class PageResponseReturnType:
  response: requests.Response
  soup: BeautifulSoup


def get_page_response_with_soup(url: str, *, query_params: Optional[dict[Any, Any]] = None, header: Optional[HeaderType] = None) -> PageResponseReturnType:
  # 👇 verify=False로 통해 SSL 검증 무시, playwright나 selenium은 브라우저를 띄워서 사용하기 때문에 verify=False일 시 해당 library를 사용하여 우회 가능.
  response = requests.get(url, params=query_params, headers=header, verify=False)
  soup = BeautifulSoup(response.text, "html.parser")


  return PageResponseReturnType(response=response, soup=soup)




# pagination 하면서 상세 논문 페이지 크롤링

In [ ]:
from tqdm.notebook import tqdm

result = []
PAGE_NUM = int(input()) 
PAPER_COUNT_PER_PAGE = 10

# progress bar 표시
crawling_progress_bar = tqdm(total=PAGE_NUM * PAPER_COUNT_PER_PAGE, desc="잰행률", unit="paper", dynamic_ncols=True, miniters=1)

for page_count in range(1, PAGE_NUM + 1):
  paper_list_response = get_page_response_with_soup(urljoin(ROOT, PATH), query_params={
    **param_dict,
    "pageNumber": page_count
  })

  previous_link, paper_list_soup = paper_list_response.response.url, paper_list_response.soup
  paper_list_element = paper_list_soup.select(".srchResultListW > ul >  li")

  for paper_element in paper_list_element:
    title = paper_element.select_one(".title > a").get_text(strip=True)
    link = urljoin(ROOT, paper_element.select_one(".title > a").get("href"))
    # print(f"================================ visiting detail paper {title} =================================")
    paper_detail_response = get_page_response_with_soup(link, header={
      "User-Agent": "Mozilla/5.0",
      "Referer": previous_link
    })
    paper = paper_detail_response.soup
    press = paper.find("span", string="발행기관").find_next_sibling().get_text(strip=True)
    year = paper.find("span", string="발행연도").find_next_sibling().get_text(strip=True)
    # 👇 detail paper page에 따라 주제어가 있는 곳도 없는 곳도 존재하여 분기처리
    keywords = [] if not paper.find("span", string="주제어") else [keyword.strip() for keyword in paper.find("span", string="주제어").find_next_sibling().text.split(";")]

    result.append([title, link, press, year, keywords])
    crawling_progress_bar.update(1) # 👉🏻 논문 한 건 처리시마다 1씩 증가
    # print(title, link, press, year, keywords, sep="\n")
    # print("="*100)

잰행률:   0%|          | 0/40 [00:00<?, ?paper/s]

# Save to Excel

In [7]:
import pandas as pd

df = pd.DataFrame(result, columns=["제목", "링크", "발행기관", "발행연도", "키워드"])
df.to_excel("./outputs/riss_paper.xlsx", index=False)


# dataclass vs TypedDict

- TypedDict: dictionary형태의 문법과 추론을 지원하는 타입 흰트 문법
  - 함수형 문법
    - 이중 dictionary의 타입을 클래스형은 추론 못 하지만 함수형 문법은 추론 가능함.
    ```python
    HeaderType = TypedDict("HeaderType", {"User-Agent": str, "Referer": str})
    ```
  - 클래스형 문법 :
    ```python
    class HeaderType(TypedDict):
      User-Agent: str
      Referer: str
    ```

- dataclass
  - 함수형 문법: @dataclass를 이용
  - TypedDict과 다르게 HeaderType(response = ... , soup = ...) 처럼 명시적으로 타입을 지정하여 사용할 수 있음.
  ```python
    @dataclass(frozen=True)
    class PageResponseReturnType:
      response: requests.Response
      soup: BeautifulSoup
  ```
  